In [1]:
from itertools import combinations
from collections import defaultdict
from optparse import OptionParser
import pandas as pd
import sys
import time
# 基本參數

freqSet_dic = dict() # 放frequent item與出現次數

def data_transform():
    """資料變形，把同transation放在同個row """
    file = pd.read_csv("testdata2.csv",header=None)
    temp_read=[]
    tran_list=[]  # 同transation在同個row
    itemSet = set() #單一item的set
    tranID = 1
    for i in range(file.shape[0]):
        if file.iloc[i][1] != tranID :
            tran_list.append(temp_read)
            temp_read=[]
            tranID += 1
        temp = file.iloc[i][2]
        temp_read.append(temp)
        b = set()
        b.add(temp)
        itemSet.add(frozenset(b))
    tran_list.append(temp_read)    
    return tran_list,itemSet


def returnItemsWithMinSupport(itemSet, transactionList, minSupport,freqSet_dic):
        _itemSet = set()
        localSet = defaultdict(int)
        item_dic = {} # 放此輪frequent item的dictionary最後會放入freqSet_dic
        for item in itemSet:
            for transaction in transactionList:
                if item.issubset(transaction):
                    localSet[item] += 1

        for item, count in localSet.items():
                support = count
                #support = float(count)/len(transactionList)

                if support >= minSupport :
                        _itemSet.add(item)
                        item_dic[item] = count
                freqSet_dic = {**freqSet_dic,**item_dic}
        return _itemSet,freqSet_dic
    

def combineset(temp_set,size):
    '''列出所有排列組合 (item集合,幾個一組)'''
    temp_set2 = set()
    for c in combinations(temp_set , 2):
        c = set(c[0])|set(c[1])
        if(len(c)) ==size:
            temp_set2.add(frozenset(c))
    return temp_set2


def combine(temp_list, n):
    '''從set中挑出幾個items'''
    temp_list2 = []
    for c in combinations(temp_list, n):
        temp_list2.append(c)
    return temp_list2

def produceRule(subSet,originSet,minConfidence):
    for i in range(len(subSet)):
        cutList = list(subSet)
        del cutList[i]
        if len(cutList)<1:
            return            
        confidence = freqSet_dic[originSet]/freqSet_dic[frozenset(cutList)]
        if confidence>=minConfidence :
            answerList=[]
            answerList.append(frozenset(cutList))
            answerList.append(frozenset(originSet)-frozenset(cutList))
            rules_dic[tuple(answerList)] = confidence
            produceRule(frozenset(cutList),originSet,minConfidence)

In [2]:
tstart = time.time()
"""找出所有freqSet"""
tranList,itemSet = data_transform()
times=0
while True:
    times = times +1
    itemSet,freqSet_dic = returnItemsWithMinSupport(itemSet,tranList,100,freqSet_dic)
    itemSet = combineset(itemSet,times+1)
    if len(itemSet)<1 :
        break
print('Length of frequent sets',len(freqSet_dic))
print(freqSet_dic)
# for item in freqSet_dic.items():
#     print(item)


tEnd = time.time()

# print('Time',tEnd-tstart)
"""找出所有rules"""
freqSet_list=[] 
rules_dic={} # 最後結果輸出
for key,value in  freqSet_dic.items(): # 先把dictionary中的key取出放入freqSet_list
    if len(key)>1:
        freqSet_list.append(key)
freqSet_list = sorted(freqSet_list,key = lambda i:len(i),reverse=True) #依照長度做排序 方便做排列組合

# 呼叫遞迴，一次丟一個freq_set
for item in freqSet_list:
    produceRule(item,item,0.5)
    
# 印出結果
# print ('Length of Rules',len(rules_dic))
# for key,values in  rules_dic.items():
#     print (key,values)
        

Length of frequent sets 2451
{frozenset({28}): 1175, frozenset({25}): 578, frozenset({10}): 506, frozenset({18}): 412, frozenset({23}): 752, frozenset({14}): 967, frozenset({27}): 199, frozenset({24}): 301, frozenset({8}): 1180, frozenset({13}): 484, frozenset({16}): 186, frozenset({22}): 358, frozenset({11}): 1238, frozenset({17}): 1125, frozenset({9}): 966, frozenset({26}): 622, frozenset({5}): 618, frozenset({15}): 231, frozenset({12}): 509, frozenset({4}): 390, frozenset({20}): 226, frozenset({7}): 630, frozenset({3}): 1135, frozenset({29}): 954, frozenset({6}): 450, frozenset({19}): 461, frozenset({21}): 765, frozenset({0}): 598, frozenset({25, 4}): 117, frozenset({10, 13}): 114, frozenset({9, 22}): 198, frozenset({3, 4}): 234, frozenset({16, 17}): 126, frozenset({25, 7}): 219, frozenset({9, 13}): 244, frozenset({0, 29}): 318, frozenset({11, 14}): 622, frozenset({0, 18}): 130, frozenset({3, 27}): 118, frozenset({8, 3}): 719, frozenset({21, 22}): 146, frozenset({10, 12}): 137, froz